In [10]:
from comet import download_model, load_from_checkpoint

'''#model_path = download_model("Unbabel/XCOMET-XL")
#model = load_from_checkpoint(model_path)
data = [
    {
        "src": "Boris Johnson teeters on edge of favour with Tory MPs", 
        "mt": "Boris Johnson ist bei Tory-Abgeordneten völlig in der Gunst", 
        "ref": "Boris Johnsons Beliebtheit bei Tory-MPs steht auf der Kippe"
    }
]
model_output = model.predict(data, batch_size=1, gpus=1)
# Segment-level scores
print (model_output.scores)

# System-level score
print (model_output.system_score)

# Score explanation (error spans)
print (model_output.metadata.error_spans)'''

RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
cannot import name 'quantize_' from 'torchao.quantization' (/fs/classhomes/fall2024/cmsc723/c7230021/FineGrainedErrorDetection/venv/lib/python3.8/site-packages/torchao/quantization/__init__.py)

In [4]:
model_output

Prediction([('scores', [0.48201069235801697]),
            ('system_score', 0.48201069235801697),
            ('metadata',
             Prediction([('src_scores', [0.7944673895835876]),
                         ('ref_scores', [0.5329917669296265]),
                         ('unified_scores', [-0.027917219325900078]),
                         ('mqm_scores', [1.0]),
                         ('error_spans', [[]])]))])

In [15]:
#from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/wmt22-cometkiwi-da")
model = load_from_checkpoint(model_path)
data = [
    {
        "src": "The output signal provides constant sync so the display never glitches.",
        "mt": "Das Ausgangssignal bietet eine konstante Synchronisation, so dass die Anzeige nie stört."
    },
    {
        "src": "Kroužek ilustrace je určen všem milovníkům umění ve věku od 10 do 15 let.",
        "mt": "Кільце ілюстрації призначене для всіх любителів мистецтва у віці від 10 до 15 років."
    },
    {
        "src": "Mandela then became South Africa's first black president after his African National Congress party won the 1994 election.",
        "mt": "その後、1994年の選挙でアフリカ国民会議派が勝利し、南アフリカ初の黒人大統領となった。"
    }, 
    {
        "src": "Hello.",
        "mt": "Well."
    }
]
model_output = model.predict(data, batch_size=8, gpus=1)
print (model_output)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/b3a8aea5a5fc22db68a554b92b3d96eb6ea75cc9/checkpoints/model.ckpt`
Encoder model frozen.
/fs/classhomes/fall2024/cmsc723/c7230021/FineGrainedErrorDetection/venv/lib/python3.8/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
/fs/classhomes/fall2024/cmsc723/c7230021/FineGrainedErrorDetection/venv/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /fs/classhomes/fall2024/cmsc723/c7230021/FineGrained ...
GPU available

Prediction([('scores', [0.36650875210762024, 0.2604619264602661, 0.2621154189109802, 0.30554649233818054]), ('system_score', 0.2986581474542618)])


In [13]:
model_output

Prediction([('scores',
             [0.3188178837299347,
              0.367400586605072,
              0.2567918002605438,
              0.2203170657157898]),
            ('system_score', 0.2908318340778351)])

In [55]:
model.encoder.model.embeddings

XLMRobertaEmbeddings(
  (word_embeddings): Embedding(250002, 1024, padding_idx=1)
  (position_embeddings): Embedding(514, 1024, padding_idx=1)
  (token_type_embeddings): Embedding(1, 1024)
  (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [57]:
model.encoder.model.encoder.layer

ModuleList(
  (0-23): 24 x XLMRobertaLayer(
    (attention): XLMRobertaAttention(
      (self): XLMRobertaSdpaSelfAttention(
        (query): Linear(in_features=1024, out_features=1024, bias=True)
        (key): Linear(in_features=1024, out_features=1024, bias=True)
        (value): Linear(in_features=1024, out_features=1024, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): XLMRobertaSelfOutput(
        (dense): Linear(in_features=1024, out_features=1024, bias=True)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): XLMRobertaIntermediate(
      (dense): Linear(in_features=1024, out_features=4096, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): XLMRobertaOutput(
      (dense): Linear(in_features=4096, out_features=1024, bias=True)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropou

In [ ]:
from torchtune.modules.peft import LoRALinear
model.encoder.model.encoder.layer[0].attention.self.query = LoRALinear(in_dim=1024, out_dim=1024, use_bias=False, rank=8, alpha=0.1, quantize_base=True)

model.encoder.model.encoder.layer[0].attention.self.query

LoRALinear(
  (dropout): Dropout(p=0.0, inplace=False)
  (lora_a): Linear(in_features=1024, out_features=8, bias=False)
  (lora_b): Linear(in_features=8, out_features=1024, bias=False)
)

In [58]:
model.layerwise_attention.scalar_parameters

ParameterList(
    (0): Parameter containing: [torch.float32 of size 1]
    (1): Parameter containing: [torch.float32 of size 1]
    (2): Parameter containing: [torch.float32 of size 1]
    (3): Parameter containing: [torch.float32 of size 1]
    (4): Parameter containing: [torch.float32 of size 1]
    (5): Parameter containing: [torch.float32 of size 1]
    (6): Parameter containing: [torch.float32 of size 1]
    (7): Parameter containing: [torch.float32 of size 1]
    (8): Parameter containing: [torch.float32 of size 1]
    (9): Parameter containing: [torch.float32 of size 1]
    (10): Parameter containing: [torch.float32 of size 1]
    (11): Parameter containing: [torch.float32 of size 1]
    (12): Parameter containing: [torch.float32 of size 1]
    (13): Parameter containing: [torch.float32 of size 1]
    (14): Parameter containing: [torch.float32 of size 1]
    (15): Parameter containing: [torch.float32 of size 1]
    (16): Parameter containing: [torch.float32 of size 1]
    (17):

In [42]:
model.train_corr

RegressionMetrics()

In [51]:
model.val_corr

ModuleList(
  (0-2): 3 x RegressionMetrics()
)

In [46]:
model.estimator

FeedForward(
  (ff): Sequential(
    (0): Linear(in_features=1024, out_features=3072, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=3072, out_features=1024, bias=True)
    (4): Tanh()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=1024, out_features=1, bias=True)
  )
)

In [48]:
model.sentloss

MSELoss()